In [1]:
from __future__ import print_function

import os
import sys
sys.path.insert(0, os.path.abspath("../.."))

# IBM PAIRS open-source module
from ibmpairs import paw
import covid19_userLocal as covid19

import numpy
import pandas
import geopandas
from shapely import wkb, wkt

from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import pytz

In [2]:
# For Both Cases and Growth we use the same ROLLING_WINDOW 
ROLLING_WINDOW = 14
lag_growthCases = 19
lag_growthCasesStd = 4 

lag_Feature = numpy.arange(lag_growthCases-lag_growthCasesStd, lag_growthCases+lag_growthCasesStd+1, 1)
print('lag_Feature', lag_Feature)

#dt_cutoff_training_COVID = datetime(2020,5,31, tzinfo=pytz.utc)
dt_cutoff_training_COVID = datetime(2020,7,25, tzinfo=pytz.utc)
print('Training only with COVID growth data until ', dt_cutoff_training_COVID)
dt_cutoff_training_mobility = dt_cutoff_training_COVID - timedelta(days=lag_growthCases - lag_growthCasesStd)
print('Training only with Mobility data until     ', dt_cutoff_training_mobility)
dt_cutoff_min = datetime(2020, 3, 1, tzinfo=pytz.utc)
print('Considering data from                      ', dt_cutoff_min)
dt_cutoff_latest = datetime(2020,8,2, tzinfo=pytz.utc)
print('Plotting data up to                        ', dt_cutoff_latest)

data_subdirectory = 'data/csv/run98FullTrainingJHU'
if not os.path.exists(data_subdirectory):
    os.makedirs(data_subdirectory)
print('data_subdirectory                          ', data_subdirectory)

lag_Feature [15 16 17 18 19 20 21 22 23]
Training only with COVID growth data until  2020-07-25 00:00:00+00:00
Training only with Mobility data until      2020-07-10 00:00:00+00:00
Considering data from                       2020-03-01 00:00:00+00:00
Plotting data up to                         2020-08-02 00:00:00+00:00
data_subdirectory                           data/csv/run98FullTrainingJHU


In [3]:
# One-time calculation
"""
# Local Polygons
df_region = pandas.read_csv('data/local_polygons.csv', usecols=['id', 'name', 'poly'])

df_region['poly'] = df_region['poly'].apply(lambda x: wkb.loads(x, hex=True))
df_region = df_region.rename(columns={'id': 'pairs_id'})
df_region = geopandas.GeoDataFrame(df_region, geometry='poly')

# We need County and State columns later on
new = df_region['name'].str.split('.', expand=True)
df_region['County'] = new[1]
df_region['State'] = new[0]

# Write it out without the wkb.loads
df_region_csv = df_region.copy()
del df_region_csv['poly']
df_tmp = pandas.read_csv('data/local_polygons.csv')[['id', 'poly']]
df_tmp = df_tmp.rename(columns={'id': 'pairs_id'})
df_region_csv = pandas.merge(df_region_csv, df_tmp, on='pairs_id', how='left')
df_region_csv.to_csv('data/df_region.csv', index=None)

df_region.tail()
"""

"\n# Local Polygons\ndf_region = pandas.read_csv('data/local_polygons.csv', usecols=['id', 'name', 'poly'])\n\ndf_region['poly'] = df_region['poly'].apply(lambda x: wkb.loads(x, hex=True))\ndf_region = df_region.rename(columns={'id': 'pairs_id'})\ndf_region = geopandas.GeoDataFrame(df_region, geometry='poly')\n\n# We need County and State columns later on\nnew = df_region['name'].str.split('.', expand=True)\ndf_region['County'] = new[1]\ndf_region['State'] = new[0]\n\n# Write it out without the wkb.loads\ndf_region_csv = df_region.copy()\ndel df_region_csv['poly']\ndf_tmp = pandas.read_csv('data/local_polygons.csv')[['id', 'poly']]\ndf_tmp = df_tmp.rename(columns={'id': 'pairs_id'})\ndf_region_csv = pandas.merge(df_region_csv, df_tmp, on='pairs_id', how='left')\ndf_region_csv.to_csv('data/df_region.csv', index=None)\n\ndf_region.tail()\n"

In [4]:
# Get the region data (county ids, names, and polygons)
df_region = pandas.read_csv('data/df_region.csv')
# Make a copy in the specific subfolder
df_region.to_csv(os.path.join(data_subdirectory, 'df_region.csv'), index=None)
# Read back
df_region = pandas.read_csv(os.path.join(data_subdirectory, 'df_region.csv'))
df_region['poly'] = df_region['poly'].apply(lambda x: wkb.loads(x, hex=True))
df_region = geopandas.GeoDataFrame(df_region, geometry='poly')

df_region.tail()

,pairs_id,name,County,State,poly
3136,53137,Wyoming.Niobrara,Niobrara,Wyoming,POLYGON ((-104.0591572933003 43.47913436795783...
3137,53138,Wyoming.Converse,Converse,Wyoming,"POLYGON ((-104.900693294084 42.61192936715017,..."
3138,53139,Wyoming.Goshen,Goshen,Wyoming,POLYGON ((-104.0535132932951 41.99981536658019...
3139,53140,Wyoming.Platte,Platte,Wyoming,POLYGON ((-104.6506362938513 41.65636536626036...
3140,53141,Wyoming.Laramie,Laramie,Wyoming,"POLYGON ((-104.055500293297 41.56422236617438,..."


In [5]:
# Query Local COVID-19 Cases
coronaQueryLocal = covid19.query_local(layerID='P567C6007') #JHU
df_local_covid = coronaQueryLocal.vdf[['timestamp', 'pairs_id', 'State', 'County', 'Value']]
df_local_covid = df_local_covid.rename(columns={'Value': 'Cases'})
df_local_covid['pairs_id'] = df_local_covid['pairs_id'].astype(int)
df_local_covid = df_local_covid[df_local_covid['timestamp']<=dt_cutoff_latest].reset_index(drop=True)

df_local_covid = pandas.merge(df_local_covid, df_region[['pairs_id']], on='pairs_id').reset_index()

# Make a copy in the specific subfolder
df_local_covid.to_csv(os.path.join(data_subdirectory, 'df_local_covid.csv'), index=None)

df_local_covid.tail()

No handlers could be found for logger "ibmpairs.paw"


,index,timestamp,pairs_id,State,County,Cases
604654,604654,2020-07-29 00:00:00+00:00,52569,Texas,Cottle,9
604655,604655,2020-07-30 00:00:00+00:00,52569,Texas,Cottle,11
604656,604656,2020-07-31 00:00:00+00:00,52569,Texas,Cottle,14
604657,604657,2020-08-01 00:00:00+00:00,52569,Texas,Cottle,14
604658,604658,2020-08-02 00:00:00+00:00,52569,Texas,Cottle,14


In [6]:
# Read back covid data from csv (raw cumulative cases)
df_local_covid = pandas.read_csv(os.path.join(data_subdirectory, 'df_local_covid.csv'))
df_local_covid['timestamp'] = pandas.to_datetime(df_local_covid['timestamp'])
df_local_covid.tail()

,index,timestamp,pairs_id,State,County,Cases
604654,604654,2020-07-29 00:00:00+00:00,52569,Texas,Cottle,9
604655,604655,2020-07-30 00:00:00+00:00,52569,Texas,Cottle,11
604656,604656,2020-07-31 00:00:00+00:00,52569,Texas,Cottle,14
604657,604657,2020-08-01 00:00:00+00:00,52569,Texas,Cottle,14
604658,604658,2020-08-02 00:00:00+00:00,52569,Texas,Cottle,14


In [7]:
# Query Local Mobility (Descartes lab median of max mobility)
mobilityQueryLocal = covid19.query_local(layerID='P612C6303')
df_local_mobility = mobilityQueryLocal.vdf[['timestamp', 'pairs_id', 'State', 'County', 'Value']]
df_local_mobility = df_local_mobility.rename(columns={'Value': 'Mobility'})
df_local_mobility['pairs_id'] = df_local_mobility['pairs_id'].astype(int)
df_local_mobility = df_local_mobility[df_local_mobility['timestamp']<=dt_cutoff_latest].reset_index(drop=True)

# Make a copy in the specific subfolder
df_local_mobility.to_csv(os.path.join(data_subdirectory, 'df_local_mobility.csv'), index=None)

df_local_mobility.tail()

,timestamp,pairs_id,State,County,Mobility
404375,2020-08-02 00:00:00+00:00,50662,Illinois,Clark,9.719
404376,2020-08-02 00:00:00+00:00,52588,Texas,Coleman,6.515
404377,2020-08-02 00:00:00+00:00,51082,Kentucky,Lawrence,1.596
404378,2020-08-02 00:00:00+00:00,51880,New_York,Westchester,0.613
404379,2020-08-02 00:00:00+00:00,51101,Kentucky,Pulaski,5.436


In [8]:
# Read back mobility data (Descartes lab median of max mobility)
df_local_mobility = pandas.read_csv(os.path.join(data_subdirectory, 'df_local_mobility.csv'))
df_local_mobility['timestamp'] = pandas.to_datetime(df_local_mobility['timestamp'])
df_local_mobility.tail()

,timestamp,pairs_id,State,County,Mobility
404375,2020-08-02 00:00:00+00:00,50662,Illinois,Clark,9.719
404376,2020-08-02 00:00:00+00:00,52588,Texas,Coleman,6.515
404377,2020-08-02 00:00:00+00:00,51082,Kentucky,Lawrence,1.596
404378,2020-08-02 00:00:00+00:00,51880,New_York,Westchester,0.613
404379,2020-08-02 00:00:00+00:00,51101,Kentucky,Pulaski,5.436


In [9]:
# Unstacking COVID19 and mobility

# Unstack the COVID19 data and first derivative (new cases)
df_unstacked = df_local_covid.copy()
del df_unstacked['State']
del df_unstacked['County']

df_unstacked = df_unstacked.set_index(['timestamp', 'pairs_id']).unstack().reset_index().sort_values(by='timestamp').set_index('timestamp')
df_unstacked = df_unstacked.swaplevel(axis=1)
df_unstacked = df_unstacked.replace(0, numpy.nan)

# Replace values where no change with nan so that the daily numbers make sense when reporting only every couple of days
# (also replace values with negative change)
df_unstacked[df_unstacked.diff()<=0] = numpy.nan

# Interpolate
df_unstacked = df_unstacked.interpolate(method='linear', limit_area='inside')

# New Local Cases (1st derivative)
df_new = df_unstacked.diff()


# Unstack the mobility data
df_m_unstacked = df_local_mobility.copy()
del df_m_unstacked['State']
del df_m_unstacked['County']

df_m_unstacked = df_m_unstacked.set_index(['timestamp', 'pairs_id']).unstack().reset_index().sort_values(by='timestamp').set_index('timestamp')
df_m_unstacked = df_m_unstacked.swaplevel(axis=1)

# Erase high-value mobility outliers >100miles before taking the rolling mean
df_m_unstacked = df_m_unstacked.clip(upper=100)

# Interpolate
df_m_unstacked = df_m_unstacked.interpolate(method='linear', limit_area='inside')

df_m_unstacked.tail()


# Stack and merge in order to fill in nan at all missing combinations
df_stacked = pandas.merge(df_new.stack(level='pairs_id').reset_index(),
                          df_m_unstacked.stack(level='pairs_id').reset_index(),
                          on=['timestamp', 'pairs_id'],
                          how='outer'
                         )
df_stacked['pairs_id'] = df_stacked['pairs_id'].astype(int)

# Unstack again
df_stacked = df_stacked.set_index(['timestamp', 'pairs_id']).unstack().reset_index().sort_values(
    by='timestamp').set_index('timestamp')

df_new = df_stacked[['Cases']].swaplevel(axis=1)
df_m_unstacked = df_stacked[['Mobility']].swaplevel(axis=1)

df_new.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,...,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,NaN,33.0,56.0,19.0,6.0,33.0,12.0,8.0,11.0,21.0,...,2.0,1.0,1.0,2.0,6.0,NaN,0.083333,2.00,NaN,10.0
2020-07-30 00:00:00+00:00,NaN,37.0,157.0,57.0,14.0,67.0,27.0,14.0,10.0,30.0,...,2.0,4.0,1.0,3.0,3.0,NaN,0.083333,0.25,NaN,7.0
2020-07-31 00:00:00+00:00,NaN,29.0,62.0,34.0,14.0,31.0,18.0,13.0,7.0,29.0,...,7.0,1.0,4.0,1.0,3.0,NaN,0.083333,0.25,NaN,7.0
2020-08-01 00:00:00+00:00,NaN,9.0,97.0,9.0,3.0,27.0,3.0,5.0,8.0,34.0,...,5.0,0.5,3.0,1.0,2.0,NaN,0.083333,0.25,NaN,6.0
2020-08-02 00:00:00+00:00,NaN,28.0,115.0,26.0,8.0,50.0,19.0,7.0,2.0,33.0,...,3.0,0.5,NaN,2.0,5.0,NaN,0.083333,0.25,NaN,11.0


In [10]:
# FIPS codes to pairs_id
df_fips = pandas.read_csv('data/County_PAIRS_FIPS.csv', dtype = {'FIPS' : 'string'})

# Make a copy in the specific subfolder
df_fips.to_csv(os.path.join(data_subdirectory, 'County_PAIRS_FIPS.csv'), index=None)

# Read back translation FIPS codes to pairs_id
df_fips = pandas.read_csv(os.path.join(data_subdirectory, 'County_PAIRS_FIPS.csv'), dtype = {'FIPS' : 'string'})
df_fips.tail()

,pairs_id,FIPS
3137,53137,56027
3138,53138,56009
3139,53139,56015
3140,53140,56031
3141,53141,56021


In [11]:
# Census data
df_census = pandas.read_csv('data/cc-est2019-alldata.csv', dtype = {'STATE' : 'string',
                                                                    'COUNTY' : 'string'}) 
df_census['FIPS'] = df_census['STATE'] + df_census['COUNTY']

# Use only latest (2019 estimate)
df_census = df_census[df_census['YEAR']==12]
df_census.tail()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,FIPS
716371,50,56,045,Wyoming,Weston County,12,14,499,280,219,...,5,0,0,0,0,0,0,0,0,56045
716372,50,56,045,Wyoming,Weston County,12,15,352,180,172,...,2,0,0,0,0,3,0,0,0,56045
716373,50,56,045,Wyoming,Weston County,12,16,229,107,122,...,0,0,0,0,0,0,0,0,0,56045
716374,50,56,045,Wyoming,Weston County,12,17,198,82,116,...,1,0,0,1,0,0,0,0,0,56045
716375,50,56,045,Wyoming,Weston County,12,18,200,72,128,...,2,0,0,0,0,0,0,0,0,56045


In [12]:
# Absolute population numbers

df_population = df_census[df_census['AGEGRP']==0][['FIPS', 'TOT_POP']].reset_index(drop=True)  #Total population
df_population = pandas.merge(df_fips[['pairs_id', 'FIPS']], df_population, on='FIPS').drop(columns='FIPS')
df_population = df_population.rename(columns={'TOT_POP': 'population'})
# Even after removing duplicates there are two FIPS in Alaska pointing to the same pairs_id so groupby sum pairs_id
df_population = df_population.groupby('pairs_id').sum().reset_index().sort_values(by='pairs_id')
df_population.tail()

# Make a copy in the specific subfolder
df_population.to_csv(os.path.join(data_subdirectory, 'df_population.csv'), index=None)

# Read back from disk
df_population = pandas.read_csv(os.path.join(data_subdirectory, 'df_population.csv'))

df_population.tail()

,pairs_id,population
3135,53137,2356
3136,53138,13822
3137,53139,13211
3138,53140,8393
3139,53141,99500


In [13]:
# Calculate the population density (one-time calculation)
"""
import shapely.ops as ops
import pyproj
from functools import partial

def geom_area(geom):
    # Calculate area for lat-lon polygon in km2
    geom_transformed = ops.transform(
        partial(
            pyproj.transform,
            pyproj.Proj(init='EPSG:4326'),
            pyproj.Proj(
                proj='aea',
                lat_1=geom.bounds[1],
                lat_2=geom.bounds[3])),
        geom)
    return geom_transformed.area / 1e6

df_pop_density = pandas.merge(df_population, df_region[['pairs_id', 'poly']], on='pairs_id')
#df_pop_density['population_density'] = df_pop_density['population'] / df_pop_density['poly'].apply(lambda x: x.area)
df_pop_density['population_density'] = df_pop_density['population'] / df_pop_density['poly'].apply(lambda x: geom_area(x))
del df_pop_density['poly']
del df_pop_density['population']

# Write to disk
df_pop_density.to_csv('data/df_pop_density.csv', index=False)
"""

"\nimport shapely.ops as ops\nimport pyproj\nfrom functools import partial\n\ndef geom_area(geom):\n    # Calculate area for lat-lon polygon in km2\n    geom_transformed = ops.transform(\n        partial(\n            pyproj.transform,\n            pyproj.Proj(init='EPSG:4326'),\n            pyproj.Proj(\n                proj='aea',\n                lat_1=geom.bounds[1],\n                lat_2=geom.bounds[3])),\n        geom)\n    return geom_transformed.area / 1e6\n\ndf_pop_density = pandas.merge(df_population, df_region[['pairs_id', 'poly']], on='pairs_id')\n#df_pop_density['population_density'] = df_pop_density['population'] / df_pop_density['poly'].apply(lambda x: x.area)\ndf_pop_density['population_density'] = df_pop_density['population'] / df_pop_density['poly'].apply(lambda x: geom_area(x))\ndel df_pop_density['poly']\ndel df_pop_density['population']\n\n# Write to disk\ndf_pop_density.to_csv('data/df_pop_density.csv', index=False)\n"

In [14]:
# Read population density from disk
df_pop_density = pandas.read_csv('data/df_pop_density.csv')

# Make a copy in the specific subfolder
df_pop_density.to_csv(os.path.join(data_subdirectory, 'df_pop_density.csv'), index=None)

# Read back from disk
df_pop_density = pandas.read_csv(os.path.join(data_subdirectory, 'df_pop_density.csv'))
df_pop_density.tail()

,pairs_id,population_density
3135,53137,0.346710
3136,53138,1.254922
3137,53139,2.286790
3138,53140,1.519373
3139,53141,14.217309


In [15]:
# Age-related census population data

#AGEGRP
#0 = Total
#1 = Age 0 to 4 years
#2 = Age 5 to 9 years
#3 = Age 10 to 14 years
#4 = Age 15 to 19 years
#5 = Age 20 to 24 years
#6 = Age 25 to 29 years
#7 = Age 30 to 34 years
#8 = Age 35 to 39 years
#9 = Age 40 to 44 years
#10 = Age 45 to 49 years
#11 = Age 50 to 54 years
#12 = Age 55 to 59 years
#13 = Age 60 to 64 years
#14 = Age 65 to 69 years
#15 = Age 70 to 74 years
#16 = Age 75 to 79 years
#17 = Age 80 to 84 years
#18 = Age 85 years or older

df_age_20_24 = df_census[df_census['AGEGRP']==5].reset_index(drop=True)  #Age 20 to 24 years
df_age_60_64 = df_census[df_census['AGEGRP']==13].reset_index(drop=True)  #Age 60 to 64 years
df_AgeRatio = df_age_60_64[['FIPS', 'TOT_POP']].set_index(
    'FIPS') / df_age_20_24[['FIPS', 'TOT_POP']].set_index('FIPS')
df_AgeRatio = df_AgeRatio.rename(columns={'TOT_POP': 'AgeRatio'}).reset_index()

# Clip outliers
df_AgeRatio['AgeRatio'] = df_AgeRatio['AgeRatio'].clip(lower=0.1, upper=20)
df_AgeRatio['LogAgeRatio'] = numpy.log10(df_AgeRatio['AgeRatio'])

df_AgeRatio = pandas.merge(df_fips[['pairs_id', 'FIPS']], df_AgeRatio, on='FIPS').drop(columns='FIPS')
# Even after removing duplicates there are two FIPS in Alaska pointing to the same pairs_id so groupby mean pairs_id
df_AgeRatio = df_AgeRatio.groupby('pairs_id').mean().reset_index().sort_values(by='pairs_id')

# Make a copy in the specific subfolder
df_AgeRatio.to_csv(os.path.join(data_subdirectory, 'df_AgeRatio.csv'), index=None)

# Read back from disk
df_AgeRatio = pandas.read_csv(os.path.join(data_subdirectory, 'df_AgeRatio.csv'))

df_AgeRatio.tail()

,pairs_id,AgeRatio,LogAgeRatio
3135,53137,1.466165,0.166183
3136,53138,1.474648,0.168688
3137,53139,1.263804,0.101680
3138,53140,1.867769,0.271323
3139,53141,0.951245,-0.021707


In [16]:
def process_covid_data(df_new, df_population, rolling_window_log, rolling_window_growth):
    """
    :df_new:    DataFrame with the new cases and fatalities
    
    Returns: df_log_new (Semi-log cleaned up daily cases and fatalities)
    Returns: df_log_new_rolling (Semi-log cleaned up daily data with 5 day rolling mean)
    Returns: df_growth (Growth in daily cases and fatalities)
    Returns: df_new_rolling_scaled (Daily cases and fatalities cleand up, rolling mean, normalized by 100K population)
    """
    # Build a semi-log version of the data and clean up
    df_log_new = df_new.copy()
    df_log_new[df_log_new<=0] = numpy.nan
    df_log_new = df_log_new.apply(lambda x: numpy.log(x))
    df_log_new = df_log_new.replace([numpy.inf, -numpy.inf], numpy.nan)
    
    # Remove outliers (non-symmetrical so that we don't erase too many valid high-value outliers)
    df_log_new[(df_log_new < df_log_new.rolling(3, center=True).mean() - 0.8) | 
               ((df_log_new > df_log_new.rolling(3, center=True).mean() + 1.2) & (df_log_new > 4))] = numpy.nan
    
    # Interpolate to fill in missing values
    df_log_new = df_log_new.interpolate(method='linear', limit_area='inside')
    
    # Rolling Mean
    df_log_new_rolling = df_log_new.rolling(rolling_window_log, min_periods=1).mean()
    
    # Growth rate (don't use min_periods=1 because it generates too many outliers)
    df_growth = df_log_new_rolling.diff()

    # Mask bad growth values due to derivatives of small numbers
    SMALL_VALUE = -1
    df_growth[df_log_new<=SMALL_VALUE] = numpy.nan

    # Interpolate inside to fill in missing values
    df_growth = df_growth.interpolate(method='linear', limit_area='inside')
    
    # Filling outside nans with zero
    df_growth = df_growth.fillna(0)
    
    # Apply rolling mean for growth
    df_growth = df_growth.rolling(rolling_window_growth, min_periods=1).mean()
    
    # Scale by 100K population
    df_new_rolling_scaled = numpy.exp(df_log_new_rolling)
    for pairs_id in df_log_new_rolling.columns.get_level_values('pairs_id').unique():
        df_new_rolling_scaled[pairs_id] = df_new_rolling_scaled[pairs_id] \
            * 100000. / df_population[df_population['pairs_id']==pairs_id]['population'].values[0]

    return df_log_new, df_log_new_rolling, df_growth, df_new_rolling_scaled

In [17]:
# Process the Covid data 
df_log_new, df_log_new_rolling, df_growth, df_new_rolling_scaled\
    = process_covid_data(df_new.swaplevel(axis=1)[['Cases']].swaplevel(axis=1),
                         df_population,
                         rolling_window_log=ROLLING_WINDOW, 
                         rolling_window_growth=ROLLING_WINDOW)

df_log_new_rolling.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,...,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases,Cases
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,NaN,3.266246,4.681551,3.272141,1.863092,3.600510,2.364528,2.263144,1.528152,2.580163,...,0.449398,0.779226,0.520930,0.666531,1.700903,NaN,-2.071996,-1.922198,-3.044522,1.761523
2020-07-30 00:00:00+00:00,NaN,3.300205,4.737235,3.313378,1.973123,3.665428,2.443001,2.287177,1.504118,2.577304,...,0.391474,0.878247,0.619951,0.745003,1.779375,NaN,-2.121507,-1.922198,-3.044522,1.851006
2020-07-31 00:00:00+00:00,NaN,3.279043,4.663117,3.335341,1.990349,3.617079,2.447083,2.256407,1.528152,2.587906,...,0.480957,0.878247,0.718972,0.745003,1.758826,NaN,-2.171017,-1.922198,-3.044522,1.841468
2020-08-01 00:00:00+00:00,NaN,3.200571,4.620969,3.244733,1.994397,3.580591,2.462037,2.144363,1.548701,2.711806,...,0.546407,0.878247,0.797444,0.666531,1.651392,NaN,-2.249490,-1.849941,-3.044522,1.854491
2020-08-02 00:00:00+00:00,NaN,3.195643,4.607430,3.244733,2.142928,3.593045,2.515410,2.126412,1.548701,2.768124,...,0.674389,0.878247,0.858786,0.602539,1.613614,NaN,-2.327962,-1.777683,-3.044522,1.886055


In [18]:
# Mobility rolling mean
df_Mobility_rolling = df_m_unstacked.rolling(ROLLING_WINDOW, min_periods=1).mean()

# LogMobility rolling mean
df_LogMobility_rolling = df_Mobility_rolling.swaplevel(axis=1).rename(
    columns={'Mobility': 'LogMobility'}).swaplevel(axis=1).apply(lambda x: numpy.log10(x))

df_LogMobility_rolling.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,...,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility,LogMobility
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,0.948161,1.019801,0.868611,0.886595,1.157688,0.951591,1.037751,1.048395,1.017236,0.994182,...,0.987216,0.616513,1.761194,0.596840,0.654273,NaN,0.722428,0.432866,0.829042,0.656427
2020-07-30 00:00:00+00:00,0.946543,1.015240,0.864104,0.886801,1.160837,0.949355,1.036027,1.045217,1.008311,0.994018,...,0.982468,0.615093,1.759196,0.594464,0.651984,NaN,0.714659,0.417875,0.834075,0.655701
2020-07-31 00:00:00+00:00,0.943894,1.017257,0.859503,0.887119,1.163504,0.941476,1.043098,1.048733,1.006851,0.994550,...,0.983971,0.616618,1.766877,0.603191,0.653240,NaN,0.730015,0.387987,0.838237,0.656557
2020-08-01 00:00:00+00:00,0.943183,1.014880,0.857617,0.885539,1.164497,0.939801,1.045362,1.051899,1.003390,0.988874,...,0.975534,0.615371,1.748401,0.610698,0.650932,NaN,0.676125,0.373477,0.835310,0.656009
2020-08-02 00:00:00+00:00,0.940541,1.012819,0.854406,0.887513,1.166658,0.939726,1.042077,1.048922,1.008615,0.984662,...,0.961845,0.608541,1.741986,0.612193,0.642917,NaN,0.668845,0.364551,0.816629,0.650106


In [19]:
# Filter out data before dt_cutoff_min. Corona numbers are too low. 
# We are doing this here AFTER the rolling means have been applied
del df_new
del df_log_new
df_LogCases = df_log_new_rolling[df_log_new_rolling.index>=dt_cutoff_min].swaplevel(axis=1)['Cases']
del df_log_new_rolling
df_GrowthCases = df_growth[df_growth.index>=dt_cutoff_min].swaplevel(axis=1)['Cases']
del df_growth
df_CasesCapita = df_new_rolling_scaled[df_new_rolling_scaled.index>=dt_cutoff_min].swaplevel(axis=1)['Cases']
del df_new_rolling_scaled
df_Mobility = df_Mobility_rolling[df_Mobility_rolling.index>=dt_cutoff_min].swaplevel(axis=1)['Mobility']
del df_Mobility_rolling
df_LogMobility = df_LogMobility_rolling[df_LogMobility_rolling.index>=dt_cutoff_min].swaplevel(axis=1)['LogMobility']
del df_LogMobility_rolling

In [20]:
# Make a copy in the specific subfolder
df_LogCases.to_csv(os.path.join(data_subdirectory, 'df_LogCases.csv'))

# Make a copy in the specific subfolder
df_GrowthCases.to_csv(os.path.join(data_subdirectory, 'df_GrowthCases.csv'))

# Make a copy in the specific subfolder
df_CasesCapita.to_csv(os.path.join(data_subdirectory, 'df_CasesCapita.csv'))

# Make a copy in the specific subfolder
df_Mobility.to_csv(os.path.join(data_subdirectory, 'df_Mobility.csv'))

# Make a copy in the specific subfolder
df_LogMobility.to_csv(os.path.join(data_subdirectory, 'df_LogMobility.csv'))

In [21]:
# Get processed LogCases DataFrame
df_LogCases = pandas.read_csv(os.path.join(data_subdirectory, 'df_LogCases.csv'), index_col='timestamp')
df_LogCases.index = pandas.to_datetime(df_LogCases.index)
df_LogCases.columns = df_LogCases.columns.astype(int)
df_LogCases.columns.name = 'pairs_id'
df_LogCases.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,NaN,3.266246,4.681551,3.272141,1.863092,3.600510,2.364528,2.263144,1.528152,2.580163,...,0.449398,0.779226,0.520930,0.666531,1.700903,NaN,-2.071996,-1.922198,-3.044522,1.761523
2020-07-30 00:00:00+00:00,NaN,3.300205,4.737235,3.313378,1.973123,3.665428,2.443001,2.287177,1.504118,2.577304,...,0.391474,0.878247,0.619951,0.745003,1.779375,NaN,-2.121507,-1.922198,-3.044522,1.851006
2020-07-31 00:00:00+00:00,NaN,3.279043,4.663117,3.335341,1.990349,3.617079,2.447083,2.256407,1.528152,2.587906,...,0.480957,0.878247,0.718972,0.745003,1.758826,NaN,-2.171017,-1.922198,-3.044522,1.841468
2020-08-01 00:00:00+00:00,NaN,3.200571,4.620969,3.244733,1.994397,3.580591,2.462037,2.144363,1.548701,2.711806,...,0.546407,0.878247,0.797444,0.666531,1.651392,NaN,-2.249490,-1.849941,-3.044522,1.854491
2020-08-02 00:00:00+00:00,NaN,3.195643,4.607430,3.244733,2.142928,3.593045,2.515410,2.126412,1.548701,2.768124,...,0.674389,0.878247,0.858786,0.602539,1.613614,NaN,-2.327962,-1.777683,-3.044522,1.886055


In [22]:
# Get processed GrowthCases DataFrame
df_GrowthCases = pandas.read_csv(os.path.join(data_subdirectory, 'df_GrowthCases.csv'), index_col='timestamp')
df_GrowthCases.index = pandas.to_datetime(df_GrowthCases.index)
df_GrowthCases.columns = df_GrowthCases.columns.astype(int)
df_GrowthCases.columns.name = 'pairs_id'
df_GrowthCases.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,0.0,0.013620,-0.002651,0.043744,0.050773,0.004982,0.026041,0.009702,-0.016017,-0.012847,...,-0.017746,-0.015209,0.115924,0.045604,0.024454,0.0,3.965082e-18,0.106433,0.022505,0.013854
2020-07-30 00:00:00+00:00,0.0,0.015332,-0.000849,0.040217,0.056564,0.011495,0.032670,0.005276,-0.024125,-0.017144,...,-0.023952,0.000075,0.116213,0.044136,0.030059,0.0,3.965082e-18,0.094415,0.016879,0.029387
2020-07-31 00:00:00+00:00,0.0,0.013552,-0.006382,0.034505,0.049097,0.003883,0.032323,-0.001597,-0.019409,-0.020575,...,-0.017560,0.005680,0.117681,0.037063,0.032127,0.0,3.965082e-18,0.083172,0.012056,0.025169
2020-08-01 00:00:00+00:00,0.0,0.004948,-0.012900,0.020960,0.041174,-0.001930,0.031809,-0.017603,-0.016474,-0.012656,...,-0.016422,0.012753,0.126823,0.025853,0.016780,0.0,3.965082e-18,0.072703,0.008037,0.017888
2020-08-02 00:00:00+00:00,0.0,0.000708,-0.017295,0.011410,0.051784,-0.002590,0.033553,-0.021884,-0.012937,-0.014238,...,-0.003744,0.023363,0.125599,0.013175,0.010244,0.0,3.965082e-18,0.063008,0.004822,0.010163


In [23]:
# Get processed CasesCapita DataFrame
df_CasesCapita = pandas.read_csv(os.path.join(data_subdirectory, 'df_CasesCapita.csv'), index_col='timestamp')
df_CasesCapita.index = pandas.to_datetime(df_CasesCapita.index)
df_CasesCapita.columns = df_CasesCapita.columns.astype(int)
df_CasesCapita.columns.name = 'pairs_id'
df_CasesCapita.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,NaN,26.500280,28.944677,47.732184,19.571216,30.595919,33.923278,32.358079,19.508445,25.567139,...,7.749273,2.729575,11.375624,5.008923,12.939305,NaN,0.911114,1.107298,0.567366,5.850551
2020-07-30 00:00:00+00:00,NaN,27.415653,30.602160,49.741654,21.847613,32.648043,36.692551,33.145185,19.045173,25.494155,...,7.313160,3.013695,12.559707,5.417818,13.995585,NaN,0.867102,1.107298,0.567366,6.398214
2020-07-31 00:00:00+00:00,NaN,26.841583,28.416007,50.846225,22.227217,31.107093,36.842663,32.140832,19.508445,25.765868,...,7.997736,3.013695,13.867041,5.417818,13.710928,NaN,0.825217,1.107298,0.567366,6.337479
2020-08-01 00:00:00+00:00,NaN,24.815786,27.243212,46.441708,22.317363,29.992529,37.397746,28.734063,19.913465,29.164457,...,8.538693,3.013695,14.999055,5.008923,12.314274,NaN,0.762936,1.190270,0.567366,6.420551
2020-08-02 00:00:00+00:00,NaN,24.693793,26.876860,46.441708,25.891029,30.368386,39.447982,28.222859,19.913465,30.854084,...,9.704510,3.013695,15.947930,4.698435,11.857741,NaN,0.705355,1.279459,0.567366,6.626436


In [24]:
# Get processed Mobility DataFrame
df_Mobility = pandas.read_csv(os.path.join(data_subdirectory, 'df_Mobility.csv'), index_col='timestamp')
df_Mobility.index = pandas.to_datetime(df_Mobility.index)
df_Mobility.columns = df_Mobility.columns.astype(int)
df_Mobility.columns.name = 'pairs_id'
df_Mobility.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,8.874857,10.466500,7.389429,7.701857,14.377643,8.945214,10.908143,11.178786,10.404857,9.866929,...,9.709929,4.135357,57.702357,3.952214,4.511000,NaN,5.277500,2.709357,6.745929,4.533429
2020-07-30 00:00:00+00:00,8.841857,10.357143,7.313143,7.705500,14.482286,8.899286,10.864929,11.097286,10.193214,9.863214,...,9.604357,4.121857,57.437571,3.930643,4.487286,NaN,5.183929,2.617429,6.824571,4.525857
2020-07-31 00:00:00+00:00,8.788071,10.405357,7.236071,7.711143,14.571500,8.739286,11.043286,11.187500,10.159000,9.875286,...,9.637643,4.136357,58.462429,4.010429,4.500286,NaN,5.370500,2.443357,6.890286,4.534786
2020-08-01 00:00:00+00:00,8.773714,10.348571,7.204714,7.683143,14.604857,8.705643,11.101000,11.269357,10.078357,9.747071,...,9.452214,4.124500,56.027500,4.080357,4.476429,NaN,4.743786,2.363071,6.844000,4.529071
2020-08-02 00:00:00+00:00,8.720500,10.299571,7.151643,7.718143,14.677714,8.704143,11.017357,11.192357,10.200357,9.653000,...,9.158929,4.060143,55.206000,4.094429,4.394571,NaN,4.664929,2.315000,6.555857,4.467929


In [25]:
# Get processed LogMobility DataFrame
df_LogMobility = pandas.read_csv(os.path.join(data_subdirectory, 'df_LogMobility.csv'), index_col='timestamp')
df_LogMobility.index = pandas.to_datetime(df_LogMobility.index)
df_LogMobility.columns = df_LogMobility.columns.astype(int)
df_LogMobility.columns.name = 'pairs_id'
df_LogMobility.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,0.948161,1.019801,0.868611,0.886595,1.157688,0.951591,1.037751,1.048395,1.017236,0.994182,...,0.987216,0.616513,1.761194,0.596840,0.654273,NaN,0.722428,0.432866,0.829042,0.656427
2020-07-30 00:00:00+00:00,0.946543,1.015240,0.864104,0.886801,1.160837,0.949355,1.036027,1.045217,1.008311,0.994018,...,0.982468,0.615093,1.759196,0.594464,0.651984,NaN,0.714659,0.417875,0.834075,0.655701
2020-07-31 00:00:00+00:00,0.943894,1.017257,0.859503,0.887119,1.163504,0.941476,1.043098,1.048733,1.006851,0.994550,...,0.983971,0.616618,1.766877,0.603191,0.653240,NaN,0.730015,0.387987,0.838237,0.656557
2020-08-01 00:00:00+00:00,0.943183,1.014880,0.857617,0.885539,1.164497,0.939801,1.045362,1.051899,1.003390,0.988874,...,0.975534,0.615371,1.748401,0.610698,0.650932,NaN,0.676125,0.373477,0.835310,0.656009
2020-08-02 00:00:00+00:00,0.940541,1.012819,0.854406,0.887513,1.166658,0.939726,1.042077,1.048922,1.008615,0.984662,...,0.961845,0.608541,1.741986,0.612193,0.642917,NaN,0.668845,0.364551,0.816629,0.650106


In [26]:
#One-time calculation
"""
# Pairwise distance between county centroids

def haversine(lat1,lon1,lat2,lon2):
    #This uses the ‘haversine’ formula to calculate the great-circle distance between two points – that is, 
    #the shortest distance over the earth’s surface – giving an ‘as-the-crow-flies’ distance between the points 
    #(ignoring any hills they fly over, of course!).
    #Haversine
    #formula:    a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
    #c = 2 ⋅ atan2( √a, √(1−a) )
    #d = R ⋅ c
    #where   φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
    #note that angles need to be in radians to pass to trig functions!
    R = 6371.0088 #km
    lat1,lon1,lat2,lon2 = map(numpy.radians, [lat1,lon1,lat2,lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = numpy.sin(dlat/2)**2 + numpy.cos(lat1) * numpy.cos(lat2) * numpy.sin(dlon/2) **2
    c = 2 * numpy.arctan2(a**0.5, (1-a)**0.5)
    d = R * c
    return round(d,4)

haversine_vec = numpy.vectorize(haversine)

# Get the polygon centroids
df_region['centroid'] = df_region['poly'].apply(lambda x: x.centroid)

# Extract the lats and lons of the centroids
df_centroids = df_region[['pairs_id', 'centroid']].set_index('pairs_id')
lons = df_centroids['centroid'].apply(lambda x: x.coords.xy[0][0]).values
lats = df_centroids['centroid'].apply(lambda x: x.coords.xy[1][0]).values
df_centroids.tail()

# Empty array to hold the results of pointwise Haversine
distance = numpy.zeros((len(lons), len(lons)))
distance[:] = numpy.nan

# Haversine distance
for i, (lon1, lat1) in enumerate(zip(lons, lats)):
    distance[i, :i+1] = haversine_vec(lats[:i+1], lons[:i+1], lat1, lon1)
    
# Numpy to Pandas and filling in nan
df_distance = pandas.DataFrame(distance)
df_distance = df_distance.fillna(pandas.DataFrame(distance.T)) # Since we only calculated half the values
df_distance.columns = list(df_centroids.index)
df_distance.index = list(df_centroids.index)

# Write to disk
df_distance.to_csv('data/df_distance.csv')
"""

"\n# Pairwise distance between county centroids\n\ndef haversine(lat1,lon1,lat2,lon2):\n    #This uses the \xe2\x80\x98haversine\xe2\x80\x99 formula to calculate the great-circle distance between two points \xe2\x80\x93 that is, \n    #the shortest distance over the earth\xe2\x80\x99s surface \xe2\x80\x93 giving an \xe2\x80\x98as-the-crow-flies\xe2\x80\x99 distance between the points \n    #(ignoring any hills they fly over, of course!).\n    #Haversine\n    #formula:    a = sin\xc2\xb2(\xce\x94\xcf\x86/2) + cos \xcf\x861 \xe2\x8b\x85 cos \xcf\x862 \xe2\x8b\x85 sin\xc2\xb2(\xce\x94\xce\xbb/2)\n    #c = 2 \xe2\x8b\x85 atan2( \xe2\x88\x9aa, \xe2\x88\x9a(1\xe2\x88\x92a) )\n    #d = R \xe2\x8b\x85 c\n    #where   \xcf\x86 is latitude, \xce\xbb is longitude, R is earth\xe2\x80\x99s radius (mean radius = 6,371km);\n    #note that angles need to be in radians to pass to trig functions!\n    R = 6371.0088 #km\n    lat1,lon1,lat2,lon2 = map(numpy.radians, [lat1,lon1,lat2,lon2])\n\n    dlat = la

In [27]:
# Read county-to-county distance matrix from disk
df_distance = pandas.read_csv('data/df_distance.csv', index_col=0)

# Make a copy in the specific subfolder
df_distance.to_csv(os.path.join(data_subdirectory, 'df_distance.csv'))

# Read back from disk
df_distance = pandas.read_csv('data/df_distance.csv', index_col=0)
df_distance.columns = list(df_distance.columns.astype(int))

df_distance.tail()

,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
53137,1709.1757,1764.0542,1799.2760,1711.8327,1759.5311,1797.4661,1726.6741,1744.9659,1780.3946,1838.8778,...,537.8090,188.8930,252.2302,186.6679,394.1078,0.0000,84.3556,108.1013,110.2552,195.5690
53138,1780.9233,1836.4976,1872.1593,1782.8840,1830.8856,1869.2925,1797.0662,1814.5783,1850.8175,1912.4664,...,456.1365,104.8634,184.0870,147.6388,313.7357,84.3556,0.0000,136.4658,103.3369,197.3083
53139,1654.5569,1710.9309,1747.1261,1655.7203,1704.0235,1742.9476,1669.1834,1685.8989,1722.9272,1788.3516,...,522.4967,222.7925,218.0869,123.5949,378.3107,108.1013,136.4658,0.0000,50.9293,91.5573
53140,1703.1532,1759.7490,1796.0823,1704.0643,1752.4568,1791.5351,1717.2767,1733.6882,1771.0079,1837.5310,...,473.1097,176.2099,169.8535,82.3576,328.5583,110.2552,103.3369,50.9293,0.0000,94.9014
53141,1648.2945,1705.9803,1743.0285,1647.9589,1696.7228,1736.5218,1659.9883,1675.0468,1713.5972,1785.7241,...,489.8743,253.5530,191.8369,94.6943,351.6616,195.5690,197.3083,91.5573,94.9014,0.0000


In [28]:
# Diffusion Term: Model the spread of covid19 from highly infected counties to other counties

fill_value = -4
# Log new cases
df_log_cases = df_LogCases.fillna(fill_value)
logCases = df_log_cases.values

# Not all pairs_ids are actually used in cases only these: df_log_cases.columns.values
df_distance_used = df_distance[df_log_cases.columns.values].T[df_log_cases.columns.values]
distance_used = df_distance_used.values
distance_used[distance_used==0] = numpy.nan # avoid division by zero for diagonal elements

# Clip to avoid outliers through division by small number
distance_used = numpy.clip(distance_used, a_min=10, a_max=None)

# Population (total for county)
df_pop = df_population.set_index('pairs_id').T[df_log_cases.columns.values]
pop = df_pop.values[0]

# Use 3D array for speedy calculation. Dimension 0  is time
logCases1 = logCases[:, :, None]  # other counties
pop1 = pop[None, :, None]   # other counties

#casesCapita0 = cases_capita[:, None, :]  # own county
logCases0 = logCases[:, None, :]  # own county
logCases_null = logCases0.copy()  # own county
logCases_null[:] = fill_value  # this makes sure the diagonal elements are nan

weighted_difference = ((logCases1 - logCases0) * pop1 / 1e5
                       / distance_used[None, :, :])

# Sum up the weighted differences
summed = numpy.nansum(weighted_difference, axis=1)  # sum over other counties

df_Diffusion = df_log_cases.copy() #Just to get the indices and columns right
df_Diffusion[:] = summed

# Write to disk
df_Diffusion.to_csv(os.path.join(data_subdirectory, 'df_Diffusion.csv'))

df_Diffusion.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,29.223028,0.973497,-4.799306,0.668668,6.471905,-0.263449,4.317591,4.753253,7.816713,4.004290,...,7.499114,6.124419,7.080241,6.896379,4.493931,16.560408,12.281892,12.653683,15.084083,4.551126
2020-07-30 00:00:00+00:00,29.244458,0.859914,-5.004126,0.530365,6.059648,-0.502190,4.037902,4.680786,7.928659,4.043913,...,7.619899,5.916569,6.860140,6.715877,4.322325,16.562009,12.389335,12.654449,15.084597,4.334053
2020-07-31 00:00:00+00:00,29.266631,0.961642,-4.682141,0.466066,6.008054,-0.292977,4.041243,4.813574,7.849873,4.016696,...,7.423244,5.923359,6.647555,6.722612,4.370521,16.570423,12.503238,12.663111,15.092365,4.364198
2020-08-01 00:00:00+00:00,29.259340,1.271027,-4.516966,0.810038,5.981960,-0.155882,3.975306,5.227022,7.759170,3.485776,...,7.272980,5.925535,6.476683,6.906278,4.599901,16.574686,12.675050,12.503818,15.096962,4.339761
2020-08-02 00:00:00+00:00,29.238339,1.267794,-4.484291,0.789982,5.373843,-0.228996,3.750792,5.273802,7.737230,3.226326,...,6.973019,5.917434,6.332905,7.046272,4.669267,16.568903,12.836753,12.333785,15.091130,4.260041


In [29]:
# Get processed Diffusion DataFrame
df_Diffusion = pandas.read_csv(os.path.join(data_subdirectory, 'df_Diffusion.csv'), index_col='timestamp')
df_Diffusion.index = pandas.to_datetime(df_Diffusion.index)
df_Diffusion.columns = df_Diffusion.columns.astype(int)
df_Diffusion.columns.name = 'pairs_id'
df_Diffusion.tail()

pairs_id,50001,50002,50003,50004,50005,50006,50007,50008,50009,50010,...,53132,53133,53134,53135,53136,53137,53138,53139,53140,53141
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-07-29 00:00:00+00:00,29.223028,0.973497,-4.799306,0.668668,6.471905,-0.263449,4.317591,4.753253,7.816713,4.004290,...,7.499114,6.124419,7.080241,6.896379,4.493931,16.560408,12.281892,12.653683,15.084083,4.551126
2020-07-30 00:00:00+00:00,29.244458,0.859914,-5.004126,0.530365,6.059648,-0.502190,4.037902,4.680786,7.928659,4.043913,...,7.619899,5.916569,6.860140,6.715877,4.322325,16.562009,12.389335,12.654449,15.084597,4.334053
2020-07-31 00:00:00+00:00,29.266631,0.961642,-4.682141,0.466066,6.008054,-0.292977,4.041243,4.813574,7.849873,4.016696,...,7.423244,5.923359,6.647555,6.722612,4.370521,16.570423,12.503238,12.663111,15.092365,4.364198
2020-08-01 00:00:00+00:00,29.259340,1.271027,-4.516966,0.810038,5.981960,-0.155882,3.975306,5.227022,7.759170,3.485776,...,7.272980,5.925535,6.476683,6.906278,4.599901,16.574686,12.675050,12.503818,15.096962,4.339761
2020-08-02 00:00:00+00:00,29.238339,1.267794,-4.484291,0.789982,5.373843,-0.228996,3.750792,5.273802,7.737230,3.226326,...,6.973019,5.917434,6.332905,7.046272,4.669267,16.568903,12.836753,12.333785,15.091130,4.260041


In [30]:
# df0: Mobility
df0 = df_Mobility.bfill().ffill()
# Create the multiindex
df0.columns = pandas.MultiIndex.from_product([df0.columns, ['Mobility']])
df0.columns.names = ['pairs_id', None]

# df1: LogMobility
df1 = df_LogMobility.bfill().ffill()
# Create the multiindex
df1.columns = pandas.MultiIndex.from_product([df1.columns, ['LogMobility']])
df1.columns.names = ['pairs_id', None]

# df2: GrowthCases
# Note: no .bfill().ffill() for GrowthCases. Instead fillna(0) before taking rolling mean
df2 = df_GrowthCases
# Create the multiindex
df2.columns = pandas.MultiIndex.from_product([df2.columns, ['GrowthCases']])
df2.columns.names = ['pairs_id', None]

# FEATURES
# df3: LogCases
df3 = df_LogCases.bfill().ffill()
# Create the multiindex
df3.columns = pandas.MultiIndex.from_product([df3.columns, ['LogCases']])
df3.columns.names = ['pairs_id', None]

# df4: CasesCapita
df4 = df_CasesCapita.bfill().ffill()
# Create the multiindex
df4.columns = pandas.MultiIndex.from_product([df4.columns, ['CasesCapita']])
df4.columns.names = ['pairs_id', None]

# df5: Diffusion
df5 = df_Diffusion.bfill().ffill()
# Create the multiindex
df5.columns = pandas.MultiIndex.from_product([df5.columns, ['Diffusion']])
df5.columns.names = ['pairs_id', None]

# CumCasesCapita
df_CumCasesCapita = df_CasesCapita.bfill().ffill().cumsum()
# Create the multiindex
df_CumCasesCapita.columns = pandas.MultiIndex.from_product([df_CumCasesCapita.columns, ['CumCasesCapita']])
df_CumCasesCapita.columns.names = ['pairs_id', None]

# Maximum mobililty between the data started and start of training
df_MaxMobility = df_Mobility.copy()
df_MaxMobility.loc[df_MaxMobility.index>dt_cutoff_training_COVID] = numpy.nan
df_MaxMobility = df_MaxMobility.groupby([True]*len(df_MaxMobility)).transform('max')
# Create the multiindex
df_MaxMobility.columns = pandas.MultiIndex.from_product([df_MaxMobility.columns, ['MaxMobility']])
df_MaxMobility.columns.names = ['pairs_id', None]

df_joined = df0.join(df1)
df_joined = df_joined.join(df2)
df_joined = df_joined.join(df3)
df_joined = df_joined.join(df4)
df_joined = df_joined.join(df5)
df_joined = df_joined.join(df_CumCasesCapita)
df_joined = df_joined.join(df_MaxMobility)

df_joined = df_joined.swaplevel(axis=1)

# Write to disk
df_joined.to_csv(os.path.join(data_subdirectory, 'df_joined.csv'))

df_joined.tail()

Mobility                                            \
pairs_id                      50001      50002     50003     50004      50005   
timestamp                                                                       
2020-07-29 00:00:00+00:00  8.874857  10.466500  7.389429  7.701857  14.377643   
2020-07-30 00:00:00+00:00  8.841857  10.357143  7.313143  7.705500  14.482286   
2020-07-31 00:00:00+00:00  8.788071  10.405357  7.236071  7.711143  14.571500   
2020-08-01 00:00:00+00:00  8.773714  10.348571  7.204714  7.683143  14.604857   
2020-08-02 00:00:00+00:00  8.720500  10.299571  7.151643  7.718143  14.677714   

                                                                      \
pairs_id                      50006      50007      50008      50009   
timestamp                                                              
2020-07-29 00:00:00+00:00  8.945214  10.908143  11.178786  10.404857   
2020-07-30 00:00:00+00:00  8.899286  10.864929  11.097286  10.193214   
2020-07-31 00:00:00+00:00  8.739286  11.043286  11.187500  10.159000   
2020-08-01 00:00:00+00:00  8.705643  11.101000  11.269357  10.078357   
2020-08-02 00:00:00+00:00  8.704143  11.017357  11.192357  10.200357   

                                     ... MaxMobility                       \
pairs_id                      50010  ...       53132     53133      53134   
timestamp                            ...                                    
2020-07-29 00:00:00+00:00  9.866929  ...   12.504929  4.916071  71.578571   
2020-07-30 00:00:00+00:00  9.863214  ...   12.504929  4.916071  71.578571   
2020-07-31 00:00:00+00:00  9.875286  ...   12.504929  4.916071  71.578571   
2020-08-01 00:00:00+00:00  9.747071  ...   12.504929  4.916071  71.578571   
2020-08-02 00:00:00+00:00  9.653000  ...   12.504929  4.916071  71.578571   

                                                                       \
pairs_id                    53135     53136 53137     53138     53139   
timestamp                                                               
2020-07-29 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-07-30 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-07-31 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-08-01 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   
2020-08-02 00:00:00+00:00  4.7655  5.909786   NaN  7.725833  4.243571   

                                               
pairs_id                      53140     53141  
timestamp                                      
2020-07-29 00:00:00+00:00  9.998929  4.868714  
2020-07-30 00:00:00+00:00  9.998929  4.868714  
2020-07-31 00:00:00+00:00  9.998929  4.868714  
2020-08-01 00:00:00+00:00  9.998929  4.868714  
2020-08-02 00:00:00+00:00  9.998929  4.868714  

[5 rows x 25088 columns]

In [31]:
# Get processed and joined DataFrame
df_joined = pandas.read_csv(os.path.join(data_subdirectory, 'df_joined.csv'), header=[0,1], index_col=0)
df_joined.index = pandas.to_datetime(df_joined.index)
# Change type of pairs_id in multiindex column to integer by transposing twice
df_joined = df_joined.T.reset_index()
df_joined['pairs_id'] = df_joined['pairs_id'].astype(int)
df_joined = df_joined.set_index(['level_0', 'pairs_id']).T
df_joined.columns.names = [None, 'pairs_id']
# Need to make it datetime index again  dtype='datetime64[ns, UTC]'
df_joined.index = df_joined.reset_index()['timestamp']
df_joined

Mobility                                  \
pairs_id                      50001      50002      50003     50004   
timestamp                                                             
2020-03-01 00:00:00+00:00  7.224000   9.717000   7.623000  7.120000   
2020-03-02 00:00:00+00:00  8.231000  12.095000   8.757500  7.632000   
2020-03-03 00:00:00+00:00  8.759000  12.958333   9.251667  8.020667   
2020-03-04 00:00:00+00:00  9.037750  13.501500   9.522500  8.125750   
2020-03-05 00:00:00+00:00  9.338000  13.985200   9.793600  8.313200   
2020-03-06 00:00:00+00:00  9.692000  14.616167  10.073500  8.703833   
2020-03-07 00:00:00+00:00  9.717714  14.519857  10.020000  8.956000   
2020-03-08 00:00:00+00:00  9.472125  13.995125   9.738250  8.766500   
2020-03-09 00:00:00+00:00  9.503333  14.064889   9.796444  8.741778   
2020-03-10 00:00:00+00:00  9.492200  14.111900   9.845300  8.757000   
2020-03-11 00:00:00+00:00  9.502182  14.132545   9.898091  8.743636   
2020-03-12 00:00:00+00:00  9.594000  14.236917   9.965167  8.774750   
2020-03-13 00:00:00+00:00  9.674385  14.362462  10.035692  8.850462   
2020-03-14 00:00:00+00:00  9.568571  14.134429   9.873714  8.821714   
2020-03-15 00:00:00+00:00  9.492500  13.885286   9.658857  8.738643   
2020-03-16 00:00:00+00:00  9.509786  13.682071   9.545500  8.684429   
2020-03-17 00:00:00+00:00  9.362786  13.311286   9.300929  8.552071   
2020-03-18 00:00:00+00:00  9.205286  12.843357   8.999929  8.442714   
2020-03-19 00:00:00+00:00  9.021571  12.323714   8.658071  8.284786   
2020-03-20 00:00:00+00:00  8.723071  11.714786   8.260857  8.013929   
2020-03-21 00:00:00+00:00  8.412071  11.144429   7.789571  7.607714   
2020-03-22 00:00:00+00:00  8.100214  10.691357   7.349929  7.285286   
2020-03-23 00:00:00+00:00  7.810643  10.052429   6.874429  7.019786   
2020-03-24 00:00:00+00:00  7.472500   9.380429   6.324071  6.687286   
2020-03-25 00:00:00+00:00  7.290143   8.909786   5.906286  6.466357   
2020-03-26 00:00:00+00:00  7.031286   8.345286   5.475000  6.235000   
2020-03-27 00:00:00+00:00  6.800786   7.848857   5.093643  6.001071   
2020-03-28 00:00:00+00:00  6.598000   7.473643   4.775143  5.720000   
2020-03-29 00:00:00+00:00  6.421429   7.291500   4.520786  5.516000   
2020-03-30 00:00:00+00:00  6.135429   6.878500   4.148286  5.351857   
...                             ...        ...        ...       ...   
2020-07-04 00:00:00+00:00  9.122643  11.029214   7.743143  8.363000   
2020-07-05 00:00:00+00:00  9.086857  10.853143   7.664714  8.305286   
2020-07-06 00:00:00+00:00  9.172929  10.871929   7.746643  8.431929   
2020-07-07 00:00:00+00:00  9.220429  10.912357   7.762214  8.476929   
2020-07-08 00:00:00+00:00  9.320786  11.006071   7.854286  8.617429   
2020-07-09 00:00:00+00:00  9.320214  10.911857   7.805643  8.630214   
2020-07-10 00:00:00+00:00  9.351714  11.035357   7.818786  8.721357   
2020-07-11 00:00:00+00:00  9.351643  11.012143   7.825143  8.827429   
2020-07-12 00:00:00+00:00  9.265071  10.878786   7.703286  8.741429   
2020-07-13 00:00:00+00:00  9.370143  10.957857   7.765071  8.782071   
2020-07-14 00:00:00+00:00  9.426500  11.038071   7.808643  8.753000   
2020-07-15 00:00:00+00:00  9.459500  11.064643   7.835643  8.709071   
2020-07-16 00:00:00+00:00  9.423286  11.031714   7.793357  8.623929   
2020-07-17 00:00:00+00:00  9.488714  11.025643   7.851429  8.558071   
2020-07-18 00:00:00+00:00  9.555571  11.189714   7.941857  8.551286   
2020-07-19 00:00:00+00:00  9.535143  11.226571   7.963214  8.424929   
2020-07-20 00:00:00+00:00  9.469857  11.213786   7.898143  8.345571   
2020-07-21 00:00:00+00:00  9.408643  11.122571   7.832643  8.263286   
2020-07-22 00:00:00+00:00  9.319143  10.987786   7.761286  8.139071   
2020-07-23 00:00:00+00:00  9.246571  10.925143   7.695286  8.009000   
2020-07-24 00:00:00+00:00  9.186286  10.829500   7.651214  7.882500   
2020-07-25 00:00:00+00:00  9.092929  10.643429   7.583857  7.735786   
2020-07-26 00:00:00+00:00  9.129643  10.716143   7.